<a href="https://colab.research.google.com/github/CalvHobbes/pricecomp_agents/blob/main/pydantic/pydantic_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'pydantic-ai[logfire]'

In [15]:
!pip install logfire


In [2]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# prompt: check if in colab environment and if so, fetch helper functions for tools

import sys

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py

In [1]:
from google.colab import userdata
import os

openapi_key = userdata.get('OPENAPI_KEY')
hf_token = userdata.get('HF_TOKEN')
os.environ["OPENAI_API_KEY"] = openapi_key
os.environ["LOGFIRE_TOKEN"] = userdata.get('LOGFIRE_TOKEN')


In [18]:
import logfire

logfire.configure(environment='local')

logfire.instrument_pydantic()  # Defaults to record='all'

Logfire project URL: https://logfire.pydantic.dev/calvhobbes/agents


In [19]:
instructions = '''
You are an intelligent assistant tasked with analyzing and combining product price data by executing one or more tools available to you. to run the tools you must provide them the name of the product the user is searching.. Your goal is to identify the most relevant product prices and present them in a structured format.  Execute all applicable tools.Follow these steps:

Combine Results: Analyze the product data from all websites. If product names or display names are slightly different but refer to the same product, use your judgment to group them together.

Identify the Best Prices: For each product, compare the prices across all websites and order them in ascending order (lowest price first).

Output Format: Present the results in the following structured format for each product:

Product Name: The most accurate or common name for the product.

For each store, include:

Store Display Name: The name of the store.

Product URL: A clickable link to the product page.

Price: The price of the product at that store.

Ensure the store-wise details are ordered by ascending price (cheapest first).

Handle Ambiguities: If product names or details are unclear, use context to determine if they refer to the same product. If uncertain, list them separately.

Sample Result:
Product Name: Wireless Noise-Cancelling Headphones
- Store: TechWorld, Product URL: [TechWorld Link], Price: $120
- Store: GadgetHub, Product URL: [GadgetHub Link], Price: $125
- Store: ElectroShop, Product URL: [ElectroShop Link], Price: $130

'''

In [20]:
model_id = "gpt-4o-mini"

In [21]:
from pydantic_ai import Agent, RunContext
from search_Croma import fetch_prices_from_croma
from search_reliance import fetch_prices_from_reliance

price_comp__agent_with_tools = Agent(
    model_id,
    deps_type=str,
    result_type=str,
    system_prompt=(
        'Use the `search_croma_tool`  and `search_reliance_tool` functions to determine best '
        'price for a product.'
    ),
)

@price_comp__agent_with_tools.tool
def search_croma_tool(ctx: RunContext[str],product: str) -> str:
    """
    This is a tool that searches for the user specified product on Croma and returns the name, url and price of the product in JSON

    Args:
        product: The product name to search for
    """
    return fetch_prices_from_croma(product)


@price_comp__agent_with_tools.tool
def search_reliance_tool(ctx: RunContext[str],product: str) -> str:
    """
    This is a tool that searches for the user specified product on Reliance and returns the name, url and price of the product in JSON

    Args:
        product: The product name to search for
    """
    return fetch_prices_from_reliance(product)

In [22]:
result = price_comp__agent_with_tools.run_sync("what is the best price for iphone 16 pro max")
print(result.data)

06:42:37.119 price_comp__agent_with_tools run prompt=what is the best price for iphone 16 pro max
06:42:37.120   preparing model and tools run_step=1
06:42:37.122   model request
06:42:39.744     Pydantic nullable validate_python
06:42:40.535     Pydantic nullable validate_python
06:42:40.537     Pydantic nullable validate_python
06:42:40.540     Pydantic nullable validate_python
06:42:40.549   handle model response
06:42:40.553     running tools=['search_croma_tool', 'search_reliance_tool']
06:42:40.553     Pydantic typed-dict validate_json
06:42:40.556     Pydantic typed-dict validate_json
06:42:47.996   preparing model and tools run_step=2
06:42:47.997   model request
06:42:55.959     Pydantic nullable validate_python
06:42:55.962     Pydantic nullable validate_python
06:42:55.963     Pydantic nullable validate_python
06:42:55.964     Pydantic nullable validate_python
06:42:55.966   handle model response
Here are the best prices for the iPhone 16 Pro Max from Croma and Reliance:

##

In [14]:
result = await price_comp__agent_with_tools.run('best price for samsung s25?')
print(result.data)



Here are the best prices for the Samsung Galaxy S25 from Croma and Reliance:

### Croma
1. **[SAMSUNG Galaxy S25 (12GB RAM, 256GB, Icyblue)](https://www.croma.com/samsung-galaxy-s25-5g-12gb-ram-256gb-icyblue-/p/313341)**
   - Price: ₹80,999

2. **[SAMSUNG Galaxy S25 (12GB RAM, 512GB, Icyblue)](https://www.croma.com/samsung-galaxy-s25-5g-12gb-ram-512gb-icyblue-/p/313335)**
   - Price: ₹92,999

### Reliance
1. **[Samsung Galaxy S25 5G 256 GB, 12 GB RAM, Icyblue](https://www.reliancedigital.in/samsung-galaxy-s25-5g-256-gb-12-gb-ram-icyblue-mobile-phone/p/494493901)**
   - Price: ₹80,999

2. **[Samsung Galaxy S25 5G 512 GB, 12 GB RAM, Icyblue](https://www.reliancedigital.in/samsung-galaxy-s25-5g-512-gb-12-gb-ram-icyblue-mobile-phone/p/494493907)**
   - Price: ₹92,999

### Best Price Summary:
- The **best price** for the **Samsung Galaxy S25 (256GB)** is **₹80,999** available at both **Croma** and **Reliance**.
- The **Samsung Galaxy S25 (512GB)** is **available for ₹92,999** at both stores

In [15]:
async with price_comp__agent_with_tools.run_stream('What is the cheapest iphone 13') as response:
    print(await response.get_data())

The cheapest iPhone 13 options are:

1. **Apple iPhone 13 (256GB, Alpine Green)**
   - Price: ₹43,994
   - [View Product](https://www.croma.com/apple-iphone-13-256gb-alpine-green-/p/249841)

2. **Apple iPhone 13 (256GB, Starlight White)**
   - Price: ₹43,994
   - [View Product](https://www.croma.com/apple-iphone-13-256gb-starlight-white-/p/243465)

3. **Apple iPhone 13 (256GB, Pink)**
   - Price: ₹43,994
   - [View Product](https://www.croma.com/apple-iphone-13-256gb-pink-/p/243466)

4. **Apple iPhone 13 (256GB, Midnight)**
   - Price: ₹43,994
   - [View Product](https://www.croma.com/apple-iphone-13-256gb-midnight-/p/243464)

5. **Apple iPhone 13 (256GB, Blue)**
   - Price: ₹43,994
   - [View Product](https://www.croma.com/apple-iphone-13-256gb-blue-/p/243468)

There are no listings for the iPhone 13 from Reliance at this time. All the listed options from Croma are for the 256GB variant, which is currently the best price available.


In [17]:
from pydantic_ai import Agent, Tool, RunContext


agent_b = Agent(
    model_id,
    tools=[
        Tool(fetch_prices_from_reliance, takes_ctx=False),
        Tool(fetch_prices_from_croma, takes_ctx=False),
    ],
)

In [20]:
result = agent_b.run_sync("what is the best price for samsung smart tv?")
print(result)

RunResult(_all_messages=[ModelRequest(parts=[UserPromptPart(content='what is the best price for samsung smart tv?', timestamp=datetime.datetime(2025, 2, 3, 11, 8, 23, 347696, tzinfo=datetime.timezone.utc), part_kind='user-prompt')], kind='request'), ModelResponse(parts=[ToolCallPart(tool_name='fetch_prices_from_reliance', args='{"product": "samsung smart tv"}', tool_call_id='call_iWZ5SHwtCDbLHjzLCp9WTwMC', part_kind='tool-call'), ToolCallPart(tool_name='fetch_prices_from_croma', args='{"product": "samsung smart tv"}', tool_call_id='call_3eTUxMY6j0BqbSh4mGQ1z1hq', part_kind='tool-call')], model_name='gpt-4o-mini', timestamp=datetime.datetime(2025, 2, 3, 11, 8, 24, tzinfo=datetime.timezone.utc), kind='response'), ModelRequest(parts=[ToolReturnPart(tool_name='fetch_prices_from_reliance', content=[{'name': 'Samsung 108 cm (43 inch) Wondertainment Series Full HD LED Smart TV, UA43TE50AAKXXL (Titan Gray)', 'url': 'https://www.reliancedigital.in/samsung-108-cm-43-inch-wondertainment-series-fu

Exception ignored in: <coroutine object Agent.run at 0x79002f4d2bc0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_variable_inspector.py", line 27, in run
KeyError: '__builtins__'
Exception ignored in: <coroutine object Agent.run at 0x79002f4d2bc0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_variable_inspector.py", line 27, in run
KeyError: '__builtins__'


In [21]:
print(result.data)

Here are some of the best prices for Samsung Smart TVs from Reliance Digital and Croma:

### Reliance Digital
1. [Samsung 108 cm (43 inch) Wondertainment Series Full HD LED Smart TV](https://www.reliancedigital.in/samsung-108-cm-43-inch-wondertainment-series-full-hd-led-smart-tv-ua43te50aakxxl-titan-gray-/p/492166127) - **₹28,990**
2. [Samsung 138 cm (55 Inch) 4K QLED Smart TV](https://www.reliancedigital.in/samsung-138-cm-55-inch-4k-qled-smart-tv-qa55q60daulxl-black-/p/494410289) - **₹69,990**
3. [Samsung 190.5 cm (75 inch) UHD Smart LED TV 75CU8000](https://www.reliancedigital.in/samsung-190-5-cm-75-inch-uhd-smart-led-tv-75cu8000/p/493711923) - **₹94,990**
4. [Samsung 127 cm (50 inch) UHD Smart LED TV 50CU7700](https://www.reliancedigital.in/samsung-127-cm-50-inch-uhd-smart-led-tv-50cu7700/p/493711913) - **₹39,990**

### Croma
1. [SAMSUNG Crystal 4K Vivid 108 cm (43 inch) 4K Ultra HD LED Tizen TV](https://www.croma.com/samsung-crystal-4k-vivid-108-cm-43-inch-4k-ultra-hd-led-tizen-tv-